In [1]:
import sys

from pygam import LinearGAM, s, f

import torch
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F # All functions that don't have any parameters
from torch.autograd import Variable

from sklearn.metrics.pairwise import rbf_kernel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from quantile_forest import RandomForestQuantileRegressor

from scipy.stats import iqr
import cvxpy as cp
import numpy as np
from numpy import linalg
import pandas as pd

from scipy.linalg import sqrtm


from matplotlib import pyplot as plt

import seaborn as sns
import matplotlib.patches as patches
import matplotlib.lines as lines

import warnings
random_seed = 42


if torch.cuda.is_available():
    print("GPU is available.")
    device = torch.device('cuda')
else:
    print("GPU is not available.")
    device = torch.device('cpu')

GPU is not available.


In [ ]:
def data_generation(n, beta): 
    d = beta.shape[0]
    X = np.random.uniform(-1, 1, (n, d))
    variance_function = 1 + ((X @ beta) ** 2)
    Y = X @ beta + (variance_function ** 0.5) * np.random.normal(n)
    return X, Y

In [2]:
# def data_generation_1dim(n): 
#     X = np.random.uniform(-1, 1, n)
#     variance_function = 1 + np.abs(X ** 5)
#     Y = 1 + X + (X ** 2) + (variance_function ** 0.5) * np.random.uniform(-1, 1, n)
#     return X, Y

def data_generation_1dim(n=2000):
    
    """
    i: random seed
    n: number of samples
    """
    
    X = np.random.uniform(-1, 1, n).reshape(-1, 1)
    Y = np.sqrt(1+25*np.power(X, 4))  * np.random.uniform(-1, 1, n).reshape(-1, 1)
    
    return X, Y

In [3]:
n = 2500

#gamma = np.array([-1, 0, 1, 1, 0])
X, Y = data_generation_1dim(n)
Data = np.hstack([X.reshape(-1, 1), Y.reshape(-1,1)])

#beta = np.array([1, 0, -1, 2, 0])
beta = 2

def shifted_data_generation(Data, beta, prop = 0.75):
    
    # random split the data from source into two parts: training and testing data
    np.random.shuffle(Data)
    train_idx = int(Data.shape[0] * prop)
    Dtrain, Dtest = np.split(Data, [train_idx])
    
    # creat the shift data set
    probabilities = np.exp(Dtest[:,1:] * beta) # Calculate probabilities proportional to w(x) = exp(x^T * beta)
    probabilities /= np.sum(probabilities) # Normalize probabilities
    probabilities = probabilities.reshape(-1,)
    
    sample_indices = np.random.choice(len(Dtest), size=len(Dtest), replace=True, p=probabilities)
    Dshift = Dtest[sample_indices]
    
    return Dtrain, Dtest, Dshift



In [4]:
class NN1(nn.Module):
    def __init__(self, input_size, output_size):
        super(NN1, self).__init__()
        self.fc1 = nn.Linear(input_size, 10)
        self.fc2 = nn.Linear(10, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
# 2-layer NN
class NN2(nn.Module):
    def __init__(self, input_size, output_size):
        super(NN2, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x 
    
    
    
class QuantileLoss(nn.Module):
    def __init__(self, quantile):
        super().__init__()
        self.quantile = quantile

    def forward(self, preds, target):
        assert not target.requires_grad
        assert preds.size(0) == target.size(0)
        residuals = target - preds
        return torch.max((self.quantile - 1) * residuals, self.quantile * residuals).mean()
        
        
def est_quantile(est_type,quantile,X_pre,Y_pre,X_opt,X_adj,X_t):
    """
        est_type: 
        "NN1": 1-layer NN;              "NN2": 2-layer NN; 
        "qrf": quantile regression forest;    "gb": gradient boosting
        
        quantile: the quantile we are estimating
        (X_pre,Y_pre): training data
        X_opt,X_adj,X_t: data used to predict
        output: quantile estimator Q and the prediction Q(X) 
    """
    
    if est_type == "NN1":
        torch.manual_seed(42)
        torch.cuda.manual_seed_all(42)
        
        # Convert numpy arrays to torch tensors
        X = torch.from_numpy(X_pre).float().to(device)
        Y = torch.from_numpy(Y_pre).float().to(device)
        input_size = X.shape[1]
        
        model = NN1(input_size=input_size, output_size=1).to(device)
        learning_rate = 0.001
        
        # Set loss function and optimizer
        criterion = QuantileLoss(quantile).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        # Training loop
        for epoch in range(1000):
            optimizer.zero_grad()   # zero the gradient buffers
            output = model(X)
            loss = criterion(output, Y)
            loss.backward()
            optimizer.step()    # Does the update
            
        # Predict
        X_opt = torch.from_numpy(X_opt).float().to(device)
        X_adj = torch.from_numpy(X_adj).float().to(device)
        X_t = torch.from_numpy(X_t).float().to(device)

        Q_opt = model(X_opt)
        Q_opt = Q_opt.detach().cpu().numpy().reshape(-1,1)
        Q_adj = model(X_adj)
        Q_adj = Q_adj.detach().cpu().numpy().reshape(-1,1)
        Q_t = model(X_t)
        Q_t = Q_t.detach().cpu().numpy().reshape(-1,1)
        return model, Q_opt, Q_adj, Q_t
    
    if est_type == "NN2":
        torch.manual_seed(42)
        torch.cuda.manual_seed_all(42) 
        
        # Convert numpy arrays to torch tensors
        X = torch.from_numpy(X_pre).float().to(device)
        Y = torch.from_numpy(Y_pre).float().to(device)
        input_size = X.shape[1]
        
        model = NN2(input_size=input_size, output_size=1).to(device)
        learning_rate = 0.001
        
        # Set loss function and optimizer
        criterion = QuantileLoss(quantile).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        # Training loop
        for epoch in range(1000):
            optimizer.zero_grad()   # zero the gradient buffers
            output = model(X)
            loss = criterion(output, Y)
            loss.backward()
            optimizer.step()    # Does the update
            
        # Predict
        X_opt = torch.from_numpy(X_opt).float().to(device)
        X_adj = torch.from_numpy(X_adj).float().to(device)
        X_t = torch.from_numpy(X_t).float().to(device)

        Q_opt = model(X_opt)
        Q_opt = Q_opt.detach().cpu().numpy().reshape(-1,1)
        Q_adj = model(X_adj)
        Q_adj = Q_adj.detach().cpu().numpy().reshape(-1,1)
        Q_t = model(X_t)
        Q_t = Q_t.detach().cpu().numpy().reshape(-1,1)
        return model, Q_opt, Q_adj, Q_t
    
    if est_type == "qrf":
        model = RandomForestQuantileRegressor(n_estimators = 500, random_state=random_seed)
        model.fit(X_pre, Y_pre)
        Q_opt = model.predict(X_opt,quantiles = [quantile]).reshape(-1,1)
        Q_adj = model.predict(X_adj,quantiles = [quantile]).reshape(-1,1)
        Q_t = model.predict(X_t,quantiles = [quantile]).reshape(-1,1)
        return model, Q_opt, Q_adj, Q_t
    
    
    if est_type == "gb":
        model = GradientBoostingRegressor(n_estimators=300,random_state=random_seed,loss = "quantile", alpha = quantile)
        model.fit(X_pre, Y_pre)
        Q_opt = model.predict(X_opt).reshape(-1,1)
        Q_adj = model.predict(X_adj).reshape(-1,1)
        Q_t = model.predict(X_t).reshape(-1,1)
        return model, Q_opt, Q_adj, Q_t


In [ ]:
Dtrain, Dtest, Dshift = shifted_data_generation(Data, beta)

In [53]:
def run_utopia(mc_iter):
    Dtrain, Dtest, Dshift = shifted_data_generation(Data, beta)
    
    total_attributes = Dtrain.shape[1]
    Y_train = Dtrain[:,0]
    X_train = Dtrain[:,1:]

    X_test = Dshift[:,1:]
    Y_test = Dshift[:,0]


    pre_idx = int(Dtrain.shape[0] * 0.5)
    Dtrain_pre, Dtrain_opt = np.split(Dtrain, [pre_idx])
    adj_idx = int(Dtrain_opt.shape[0] * 0.5)
    Dtrain_opt, Dtrain_adj = np.split(Dtrain_opt, [adj_idx])

    X_pre = Dtrain_pre[:,1:]
    Y_pre = Dtrain_pre[:,0]

    X_opt = Dtrain_opt[:,1:]
    Y_opt = Dtrain_opt[:,0]

    X_adj = Dtrain_adj[:,1:]
    Y_adj = Dtrain_adj[:,0]

    
    mean_model = LinearRegression()
    mean_model.fit(X_pre, Y_pre)
    
    ## change it to linear model 

    mean_est_pre = mean_model.predict(X_pre)
    mean_est_opt = mean_model.predict(X_opt)
    mean_est_adj = mean_model.predict(X_adj)

    Y_centered_squared_pre = (Y_pre - mean_est_pre)**2 
    Y_centered_squared_opt = (Y_opt - mean_est_opt)**2 
    Y_centered_squared_adj = (Y_adj - mean_est_adj)**2 
    
    quantile = [0.85,0.95,0.9,0.9]


    m1,Q1_opt,Q1_adj,Q1_t = est_quantile("NN1",quantile[0],X_pre,Y_centered_squared_pre,X_opt,X_adj,X_test) # quantile def
    m2,Q2_opt,Q2_adj,Q2_t = est_quantile("NN2",quantile[1],X_pre,Y_centered_squared_pre,X_opt,X_adj,X_test)
    m3,Q3_opt,Q3_adj,Q3_t = est_quantile("qrf",quantile[2],X_pre,Y_centered_squared_pre,X_opt,X_adj,X_test)
    m4,Q4_opt,Q4_adj,Q4_t = est_quantile("gb",quantile[3],X_pre,Y_centered_squared_pre,X_opt,X_adj,X_test)
    

    ######## Variance estimator ###########
    
    rf_var_model = RandomForestRegressor(n_estimators = 1000, random_state = 42, max_depth = 20)
    rf_var_model.fit(X_pre, Y_centered_squared_pre)
    var_hat_pre = rf_var_model.predict(X_pre)
    var_hat_adj = rf_var_model.predict(X_adj)
    var_hat_opt = rf_var_model.predict(X_opt)
    var_hat_test = rf_var_model.predict(X_test)
    
    E_opt = np.vstack((np.matrix.flatten(Q1_opt), np.matrix.flatten(Q2_opt), np.matrix.flatten(Q3_opt), np.matrix.flatten(Q4_opt), var_hat_opt))
    E_adj = np.vstack((np.matrix.flatten(Q1_adj), np.matrix.flatten(Q2_adj), np.matrix.flatten(Q3_adj), np.matrix.flatten(Q4_adj), var_hat_adj))
    E_test = np.vstack((np.matrix.flatten(Q1_t), np.matrix.flatten(Q2_t), np.matrix.flatten(Q3_t), np.matrix.flatten(Q4_t), var_hat_test))
    
    n_opt = X_opt.shape[0]
    n_adj = X_adj.shape[0]
    n_test = X_test.shape[0]

    cons_opt = np.ones(n_opt).reshape(1,-1)
    cons_adj = np.ones(n_adj).reshape(1,-1)
    cons_test = np.ones(n_test).reshape(1,-1)

    A_opt = np.vstack((E_opt,cons_opt))
    A_adj = np.vstack((E_adj,cons_adj))
    A_test = np.vstack((E_test,cons_test))

    weight = cp.Variable(A_opt.shape[0])


    constraints = [weight>=0]+[weight @ A_opt >= Y_centered_squared_opt]  ### Change the objective 
    prob = cp.Problem(cp.Minimize(cp.sum(weight @ A_test)), constraints)
    prob.solve()
    optimal_weight = weight.value

    f_hat_init_opt = optimal_weight @  A_opt
    f_hat_init_adj = optimal_weight @  A_adj
    f_hat_init_test = optimal_weight @ A_test
    
    f_hat_init_adj[f_hat_init_adj < 0] = 0
    f_hat_init_test[f_hat_init_test < 0] = 0
    
    domain_cov = np.vstack((X_pre, X_test))
    domain_idx =  np.vstack((np.zeros(X_pre.shape[0]).reshape(-1,1), np.ones(X_test.shape[0]).reshape(-1,1)))
    domain_idx = np.matrix.flatten(domain_idx)

    logistic_model = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000,random_state=0) 
    logistic_model.fit(domain_cov, domain_idx)

    prob_hat = logistic_model.predict_proba(X_adj)[:,0]
    prob_hat = np.clip(prob_hat, 0.01, 0.99)
    est_ratio = prob_hat/(1-prob_hat) #### Multiplying by the constant or normalization #####
    #est_ratio /= est_ratio.mean()
    est_ratio = est_ratio * (X_pre.shape[0]/X_test.shape[0])
    
    
    delta = 1.01
    alpha = 0.05
    stepsize = 0.005
    prop_outside = np.mean(est_ratio * ((Y_centered_squared_adj/(f_hat_init_adj + 1e-10)) > delta))
    while prop_outside <= alpha:
        delta = delta - stepsize
        prop_outside = np.mean(est_ratio * ((Y_centered_squared_adj/(f_hat_init_adj + 1e-10)) > delta))
        

    mean_est_test = mean_model.predict(X_test)
    Y_centered_squared_test = (Y_test - mean_est_test)**2 
    cov = np.mean(Y_centered_squared_test < (delta * f_hat_init_test))
    bw = 2 * np.mean((delta * f_hat_init_test) ** 0.5)
    return cov, bw

In [54]:
from joblib import Parallel, delayed
results_utopia = Parallel(n_jobs=10)(delayed(run_utopia)(mc_iter) for mc_iter in range(100))

In [57]:
coverage_utopia = np.array(results_utopia)[:,0]
bandwidth_utopia = np.array(results_utopia)[:,1]

In [58]:
print(np.median(coverage_utopia))
print(np.median(bandwidth_utopia))
print(np.median(bandwidth_utopia))
print(iqr(coverage_utopia))
print(iqr(bandwidth_utopia))

0.8855999999999999
2.0908599852972594
2.0908599852972594
0.2023999999999999
0.11267225571728678


In [42]:
coverage_utopia

array([0.9344, 0.8176, 0.6736, 0.816 , 0.7264, 0.6592, 0.7648, 0.7488,
       0.4128, 0.7616, 0.2976, 0.7136, 0.808 , 0.72  , 0.7312, 0.5312,
       0.4   , 0.8512, 0.6736, 0.6272, 0.8304, 0.7536, 0.7424, 0.6416,
       0.6656, 0.6544, 0.8608, 0.3824, 0.7888, 0.816 , 0.7824, 0.2528,
       0.632 , 0.5504, 0.7008, 0.6848, 0.7232, 0.8176, 0.2272, 0.9952,
       0.56  , 0.4   , 0.3232, 0.6208, 0.4016, 0.8976, 0.7664, 0.4464,
       0.7456, 0.8448, 0.6672, 0.8656, 0.8112, 0.608 , 0.6496, 0.6896,
       0.5504, 0.4656, 0.64  , 0.9216, 0.4032, 0.8304, 0.7776, 0.4048,
       0.7072, 0.8928, 0.8224, 0.9264, 0.8768, 0.3872, 0.44  , 0.8016,
       0.8128, 0.4688, 0.5216, 0.8384, 0.8976, 0.5408, 0.7712, 0.7664,
       0.7792, 0.7824, 0.7568, 0.5424, 0.7824, 0.6656, 0.8544, 0.7024,
       0.7872, 0.6736, 0.608 , 0.5696, 0.6368, 0.816 , 0.4208, 0.9664,
       0.4944, 0.7936, 0.768 , 0.752 ])

In [55]:
################################## Weighted Conformal Prediction ##########################

def prob_est(Dtrain, Dshift, est_type = 'lr'):
    X_S = Dtrain[:,1:]
    X_T = Dshift[:,1:]
    n_S = X_S.shape[0]
    n_T = X_T.shape[0]
    Y_S = np.zeros(n_S).reshape(-1,1)
    Y_T = np.ones(n_T).reshape(-1,1)
    X_train = np.vstack((X_S, X_T))
    Y_train = np.vstack((Y_S, Y_T))
  
    if est_type == 'lr':
        # fit logistic regression
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            logistic_model = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000,random_state=0) 
            logistic_model.fit(X_train, Y_train[:,0])

        return logistic_model
    
    if est_type == 'rf':
        # fit random forest
        rf_model = RandomForestClassifier(n_estimators = 100, random_state=random_seed)
        rf_model.fit(X_train, Y_train[:,0])

        return rf_model

    
def ratio_est(model, X):
    
        prob_1 = model.predict_proba(X)[:,1]
        prob_1 = np.clip(prob_1, 0.01, 0.99)
        est_ratio = prob_1/(1-prob_1)
        
        return est_ratio
    
def weighting_function(model, X_calib, x_test, x):
    # Define weighting function here
    # all the data should be array
    ratio_calib = ratio_est(model, X_calib).sum()
    ratio_test = ratio_est(model, x_test)
    ratio_sum = ratio_calib + ratio_test
    ratio_x = ratio_est(model, x)
    return ratio_x/ratio_sum


def weighted_quantile(values, weights, quantile):
    
    """ Compute the weighted quantile of a 1D numpy array.
    """
    values = np.array(values)
    weights = np.array(weights)
    
    sorter = np.argsort(values)
    values = values[sorter]
    weights = weights[sorter]
    cumulative_weights = np.cumsum(weights)
    if cumulative_weights[-1]>= quantile:
        idx = np.argmax(cumulative_weights >= quantile)
        return values[idx]
    else:
        return np.max(values)

    
    
def weighted_conformal_cov_width(Dtrain, Dshift, alpha=0.05):
    
    n = Dshift.shape[0]
    cov = []
    width = []
    
    
    prob_est_model = prob_est(Dtrain, Dshift, est_type = 'lr')
    
    # split weighted conformal
    D = Dtrain
    np.random.shuffle(D)
    split_idx = int(D.shape[0] * 0.5)
    D1, D2 = np.split(D, [split_idx])
    X1 = D1[:,1:]
    y1 = D1[:,0]
    X2 = D2[:,1:]
    y2 = D2[:,0]
    
    # Train the model on the first part of the data
#     model = LinearRegression()
#     model.fit(X1, y1)
    
    model = LinearRegression()
    model.fit(X1, y1)
    
    # Compute the model's predictions for the calibration set
    y1_hat = model.predict(X1)
    y_calib_pred = model.predict(X2)
            
    # Calculate the absolute errors on the calibration set
    errors_calib = np.abs(y_calib_pred - y2)
    
    est_error_1 = (y1_hat - y1) ** 2
    var_model = RandomForestRegressor(n_estimators = 1000, random_state = 42, max_depth = 20)
    var_model.fit(X1, est_error_1)
    var_hat = var_model.predict(X2)
    sd_calib_pred = var_hat ** 0.5
    errors_calib = np.abs(y_calib_pred - y2)/np.clip(sd_calib_pred, 0.001, 1e10)
    
    for i in range(n):
        x_test = Dshift[i,1:]
        x_test = np.array(x_test).reshape(1,-1)
        y_test = Dshift[i,0]
    
    
        # Calculate the weights for the calibration set
        weights_calib = weighting_function(prob_est_model, X2, x_test, X2)


        # Calculate the weighted quantile of the errors
        quantile = weighted_quantile(errors_calib, weights_calib, 1 - alpha)

        # Now for the test
        y_test_pred = model.predict(x_test)
        sd_test_pred = var_model.predict(x_test) ** 0.5
        
        width.append(2*quantile*sd_test_pred)
        cov.append((np.abs(y_test - y_test_pred)/sd_test_pred) <= quantile)

    
    return np.array(cov).mean(), np.array(width).mean()


In [56]:
alpha = 0.05
def run_weighted_conformal(n):
    Dtrain, Dtest, Dshift = shifted_data_generation(Data,beta)
    cov, bw = weighted_conformal_cov_width(Dtrain, Dshift, alpha)
    return cov, bw

from joblib import Parallel, delayed
results_wc_heterogeneous = Parallel(n_jobs=10)(delayed(run_weighted_conformal)(i) for i in range(100))

In [52]:
coverage_wc = np.array(results_wc_heterogeneous)[:,0]
bandwidth_wc = np.array(results_wc_heterogeneous)[:,1]

print(np.median(coverage_wc))
print(np.median(bandwidth_wc))
print(np.median(bandwidth_wc[coverage_wc > 0.95]))
print(iqr(coverage_wc))
print(iqr(bandwidth_wc))

0.9736
5.000832686224179
5.161716797214706
0.025999999999999912
2.273149799511586
